<a href="https://colab.research.google.com/github/kotgirep/kneighbour/blob/main/DataMiningKNeighbor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CMPE-255 Data Mining Assignment: 

Use state of art libraries for Approximate nearest neighbor search for your favorite data set

Import the required libraries

In [1]:
import pickle
#import faiss

upgrading the pandas libraries

In [2]:
pip install --upgrade pandas

In [4]:
import pandas.core.indexes as i

Writing a function to pickle and load dataset

In [12]:
def pickle_load():
    with open('my_pickle_dataset.pickle', 'rb') as f:
        pickle_data = pickle.load(f)
    return pickle_data

pickle_data = pickle_load()
pickle_vctr = pickle_data["vector"]
names = pickle_data["name"]
pickle_data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

Install faiss

In [13]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 3.1 kB/s 


In [13]:
import faiss
faiss.MatrixStats(pickle_vctr).comments.split("\n")

['analyzing 1682 vectors of size 64',
 'no NaN or Infs in data',
 'all vectors are distinct',
 'range of L2 norms=[0.747558, 1.80436] (0 null vectors)',
 'matrix contains no 0s',
 'no constant dimensions',
 'no dimension has a too large mean',
 'stddevs per dimension are in [0.112036 0.158214]',
 '']

Exhaustive Search using Faiss

In [16]:
vector_indx = faiss.IndexFlatL2(pickle_vctr.shape[1])
vector_indx.add(pickle_vctr)

In [18]:
pickle_search_vctr = pickle_vctr[90:91]
distances, indices = vector_indx.search(pickle_search_vctr, 10)

In [20]:
print(f"Approximates same television shows are {names[90]} are:\n")
print([names[j] for j in indices[0]])

Approximates same television shows are Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Sirens (1994)', 'Beauty and the Beast (1991)', 'Akira (1988)', 'Fantasia (1940)', 'Benny & Joon (1993)', 'Barbarella (1968)', "Pete's Dragon (1977)", 'James and the Giant Peach (1996)']


Updating clusters

In [22]:
quantizer = faiss.IndexFlatL2(pickle_vctr.shape[1])
update_indx = faiss.IndexIVFFlat(quantizer, 
                           pickle_vctr.shape[1], 
                           100,             
                           8)               
update_indx.train(pickle_vctr)
update_indx.add(pickle_vctr)

In [23]:
pickle_search_vctr = pickle_vctr[90:91]
distances, indices = vector_indx.search(pickle_search_vctr, 10)

In [25]:
print(f"Appropriate most same Television shows to watch {names[90]} are:\n")
print([names[i] for i in indices[0]])

Appropriate most same Television shows to watch Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Sirens (1994)', 'Beauty and the Beast (1991)', 'Akira (1988)', 'Fantasia (1940)', 'Benny & Joon (1993)', 'Barbarella (1968)', "Pete's Dragon (1977)", 'James and the Giant Peach (1996)']


Product Quantization

In [27]:
product_quantizr = faiss.IndexFlatL2(pickle_vctr.shape[1])
quantization_indx = faiss.IndexIVFPQ(product_quantizr, 
                         pickle_vctr.shape[1], 
                         100,             
                         8,                
                         8)               
quantization_indx.train(pickle_vctr)
quantization_indx.add(pickle_vctr)

In [28]:
quant_srch_vctr = pickle_vctr[90:91]
distances, indices = quantization_indx.search(quant_srch_vctr, 10)

In [29]:
print(f"Appropriate most same Television shows to watch {names[90]} are:\n")
print([names[k] for k in indices[0]])

Appropriate most same Television shows to watch Nightmare Before Christmas, The (1993) are:

['Nightmare Before Christmas, The (1993)', 'Heavy Metal (1981)', 'Pink Floyd - The Wall (1982)', 'Akira (1988)', 'Hamlet (1996)', 'Full Metal Jacket (1987)', 'Supercop (1992)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)', 'Scream of Stone (Schrei aus Stein) (1991)']


Trees and Graphs

Install the required libraries

In [31]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  # For gaussian_random_matrix.
!pip install annoy

     |████████████████████████████████| 9.8 MB 5.3 MB/s 
     |████████████████████████████████| 2.3 MB 43.0 MB/s 
     |████████████████████████████████| 62 kB 781 kB/s 
     |████████████████████████████████| 829 kB 46.1 MB/s 
     |████████████████████████████████| 151 kB 52.5 MB/s 
     |████████████████████████████████| 249 kB 44.7 MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-py3-none-any.whl size=43512 sha256=9dcfbe3b9b4d7e2ab58aa9fe607864a5980e3c0f251dd6d567a80ff7e4697add
  Stored in directory: /root/.cache/pip/wheels/bc/49/5f/fdb5b9d85055c478213e0158ac122b596816149a02d82e0ab1
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78546 sha256=65edc22030b3c8979d852f0ede5ba243ec4c8b186bf60ec1542afc4ae86cd87d
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=697ebe183eccb5e441d2f9d0682867d243d420d574

In [32]:
import annoy

In [34]:
def annoy_load():
    with open('my_pickle_dataset.pickle', 'rb') as f:
        annoy_data = pickle.load(f)
    return annoy_data

annoy_data = annoy_load()
annoy_data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

Annoy

In [41]:
class anoy_indx():
    def __init__(self, annoy_vectors, total_lbls):
        self.dimention = annoy_vectors.shape[1]
        self.annoy_vectors = annoy_vectors.astype('float32')
        self.total_lbls = total_lbls


    def build(self, total_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.annoy_vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(total_trees)
        
    def query(self, vector, k=11):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.total_lbls[l] for l in indices]

In [45]:
index = anoy_indx(annoy_data["vector"], annoy_data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [47]:
show_vctr, name_of_the_show = annoy_data['vector'][90], annoy_data['name'][90]
same_movies = '\n* '.join(index.query(show_vctr))
print(f"Appropriate same movies that have to watch {name_of_the_show} are:\n* {same_movies}")

Appropriate same movies that have to watch Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Aladdin (1992)
* Blade Runner (1982)
* Aliens (1986)
* Pink Floyd - The Wall (1982)
* Brazil (1985)
* Wrong Trousers, The (1993)
* Alien (1979)
* Return of the Jedi (1983)
* E.T. the Extra-Terrestrial (1982)
* Grand Day Out, A (1992)


Graphs

In [49]:
!pip install nmslib

     |████████████████████████████████| 13.5 MB 50 kB/s 
     |████████████████████████████████| 188 kB 33.2 MB/s 


In [51]:
import nmslib

In [54]:
def graph_load_data():
    with open('my_pickle_dataset.pickle', 'rb') as f:
        graph_data = pickle.load(f)
    return graph_data

graph_data = graph_load_data()
graph_data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.01780608, -0.14265831,  0.10308606, ...,  0.09659795,
         -0.17529577, -0.03061521],
        [-0.03357764,  0.16418771,  0.21801303, ...,  0.16502103,
         -0.09166156,  0.05047869],
        [-0.2761452 , -0.01991325, -0.04969981, ...,  0.0258275 ,
         -0.08328608, -0.0152858 ],
        ...,
        [ 0.05142734, -0.01683608, -0.20441587, ...,  0.00045828,
          0.14679626,  0.2462584 ],
        [ 0.04491899, -0.02819411, -0.09472758, ..., -0.02152078,
          0.16223577,  0.19897607],
        [ 0.02531924,  0.03099714,  0.06437534, ..., -0.07260127,
          0.0467432 ,  0.07893164]], dtype=float32)}

In [58]:
class hnsw_indx():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[z] for z in indices[0]]

In [59]:
graph_indx = hnsw_indx(graph_data["vector"], graph_data["name"])
graph_indx.build()

In [60]:
graph_vectr, show_name = graph_data['vector'][90], graph_data['name'][90]
same_shows = '\n* '.join(graph_indx.query(graph_vectr))
print(f"Appropriate similar tv shows {show_name} are:\n* {same_shows}")

Appropriate similar tv shows Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Beauty and the Beast (1991)
* Fantasia (1940)
* Heavy Metal (1981)
* Aladdin (1992)
* Snow White and the Seven Dwarfs (1937)
* Batman (1989)
* James and the Giant Peach (1996)
* Blade Runner (1982)
* Aliens (1986)


Locality Sensitive Hashing

In [66]:
class sensitivehashing():
    def __init__(self, lsh_vctrs, lsh_lbls):
        self.dimention = lsh_vctrs.shape[1]
        self.lsh_vctrs = lsh_vctrs.astype('float32')
        self.lsh_lbls = lsh_lbls


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        lsh_quant = faiss.IndexFlatL2(self.dimention)
        self.graph_indx = faiss.IndexIVFPQ(lsh_quant, self.dimention, number_of_partition, search_in_x_partitions, subvector_size)
        self.graph_indx.train(self.lsh_vctrs)
        self.graph_indx.add(self.lsh_vctrs)
        
    def query(self, lsh_vctrs, k=10):
        distances, indices = self.graph_indx.search(lsh_vctrs, k) 
        return [self.lsh_lbls[a] for a in indices[0]]

In [68]:
graph_indx = sensitivehashing(graph_data["vector"], graph_data["name"])
graph_indx.build()

In [69]:
graph_vector, name_of_the_show = graph_data['vector'][90:91], graph_data['name'][90]
Movie_exact_same_names = '\n* '.join(graph_indx.query(graph_vector))
print(f"The most similar movies to {name_of_the_show} are:\n* {Movie_exact_same_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Fantasia (1940)
* Brazil (1985)
* Monty Python's Life of Brian (1979)
* This Is Spinal Tap (1984)
* Hunt for Red October, The (1990)
* Sneakers (1992)
* Lion King, The (1994)
* Clockwork Orange, A (1971)
* Full Metal Jacket (1987)
